In [1]:
# import necessary libraries

import pandas as pd
import numpy as np
import os

In [2]:
# import all datasources and convert them into pandas dataframes

path = '/Users/ilaydabekircan/Documents/Vision Bridge/DS Assignment/AssignmentData'

csv_files = [file for file in os.listdir(path) if file.endswith('.csv')]

for file in csv_files:
    df_name = file.split('.')[0]
    globals()[df_name] = pd.read_csv(os.path.join(path, file))

In [3]:
# print names of each dataframe
df_names = [name for name, obj in globals().items() if isinstance(obj, pd.DataFrame)]
df_names

['Customers',
 'ChannelsCampaigns',
 'Products',
 'StoresCampaigns',
 'Orders',
 'OrderItems',
 'Users',
 'AudiencesCampaigns',
 'Retailers',
 'MarketingCampaigns',
 'CampaignResults',
 'Tracking',
 'Channels',
 'Audiences',
 'Transactions',
 'BehavioralData',
 'Stores']

# BUYERS

In [4]:
print('Number of customers in our dataset', Customers['CustomerID'].nunique())
print('Number of customers who bought something:', Orders['CustomerID'].nunique())
print('Number of customers who didnt buy something:', Customers['CustomerID'].nunique() - Orders['CustomerID'].nunique())

Number of customers in our dataset 5000
Number of customers who bought something: 917
Number of customers who didnt buy something: 4083


Although most of the customers didn't buy anything, we can still learn from their behaviours such as their clicks. (BehavioralData)

In [5]:
Customers['CustomerID'] = Customers['CustomerID'].astype(float)
Customers['RetailerID'] = Customers['RetailerID'].astype(float)
Customers.rename(columns = {'City': 'CustomerCity', 'State': 'CustomerState'}, inplace=True)

CustomerBuys = Customers.merge(Orders, 
                               how = 'left',
                               on = 'CustomerID')
CustomerBuys.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5083 entries, 0 to 5082
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        5083 non-null   float64
 1   RetailerID        5083 non-null   float64
 2   Name              5083 non-null   object 
 3   Email             5083 non-null   object 
 4   Phone             5083 non-null   object 
 5   CustomerCity      5083 non-null   object 
 6   CustomerState     5083 non-null   object 
 7   Country           5083 non-null   object 
 8   Gender            5083 non-null   object 
 9   Age               5083 non-null   object 
 10  EmploymentStatus  5083 non-null   object 
 11  Education         5083 non-null   object 
 12  Occupation        3677 non-null   object 
 13  Industry          3677 non-null   object 
 14  Income            5083 non-null   int64  
 15  CreatedAt         5083 non-null   object 
 16  OrderID           1000 non-null   float64


In [6]:
no_orders = CustomerBuys[CustomerBuys['OrderID'].isna()]

num_customers_no_orders = no_orders['CustomerID'].nunique()

print(f"Number of customers who didn't buy anything: {num_customers_no_orders}")


Number of customers who didn't buy anything: 4083


In [7]:
# Drop unnecessary columns, country is redundant because all of them are USA
CustomerBuys.drop(columns = ['Email', 'Phone', 'Country', 'CreatedAt'], inplace = True)
CustomerBuys.rename(columns={'Name': 'CustomerName'}, inplace=True)

In [8]:
OrderItems.drop(columns = ['OrderItemID'], inplace = True)

# price is shown for only one product so the total price must be calculated
OrderItems[OrderItems['ProductID'] == 1609]

,OrderID,ProductID,Price,Quantity
793,407,1609,23.03,3
1505,756,1609,23.03,4
1556,782,1609,23.03,2
1678,839,1609,23.03,1


In [9]:
# total price is calculated for each order id
OrderItems['TotalPrice'] = OrderItems['Price'] * OrderItems['Quantity']

OrderItems['OrderID'] = OrderItems['OrderID'].astype(float)

# products with missing price information are removed
OrderItems.dropna(subset = ['Price'], 
                  inplace = True)

OrderItems.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1919 entries, 0 to 1994
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   OrderID     1919 non-null   float64
 1   ProductID   1919 non-null   int64  
 2   Price       1919 non-null   float64
 3   Quantity    1919 non-null   int64  
 4   TotalPrice  1919 non-null   float64
dtypes: float64(3), int64(2)
memory usage: 90.0 KB


In [10]:
CustomerBuys = CustomerBuys.merge(OrderItems, 
                                  how = 'left',
                                  on = 'OrderID')
CustomerBuys.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6015 entries, 0 to 6014
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        6015 non-null   float64
 1   RetailerID        6015 non-null   float64
 2   CustomerName      6015 non-null   object 
 3   CustomerCity      6015 non-null   object 
 4   CustomerState     6015 non-null   object 
 5   Gender            6015 non-null   object 
 6   Age               6015 non-null   object 
 7   EmploymentStatus  6015 non-null   object 
 8   Education         6015 non-null   object 
 9   Occupation        4330 non-null   object 
 10  Industry          4330 non-null   object 
 11  Income            6015 non-null   int64  
 12  OrderID           1932 non-null   float64
 13  OrderDate         1932 non-null   object 
 14  Count             1932 non-null   float64
 15  TotalAmount       1932 non-null   float64
 16  Status            1932 non-null   object 


In [11]:
CustomerBuys[(CustomerBuys['TotalAmount'] != CustomerBuys['TotalPrice']) & (CustomerBuys['TotalAmount'].notna())]

,CustomerID,RetailerID,CustomerName,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,...,Income,OrderID,OrderDate,Count,TotalAmount,Status,ProductID,Price,Quantity,TotalPrice
8,9.0,40.0,Kayla Garza,Seattle city,Washington,male,25 to 34 years,employed,graduate_or_professional_degree,sales_and_office_occupations,...,61037,887.0,2024-02-07 18:10:10,2.0,5445.58,Completed,3968.0,27.59,2.0,55.18
9,9.0,40.0,Kayla Garza,Seattle city,Washington,male,25 to 34 years,employed,graduate_or_professional_degree,sales_and_office_occupations,...,61037,887.0,2024-02-07 18:10:10,2.0,5445.58,Completed,6351.0,2695.20,2.0,5390.40
21,21.0,22.0,Elizabeth Moon,San Diego city,California,male,18 to 24 years,employed,bachelors_degree,production_transportation_material_moving,...,60797,161.0,2024-10-08 10:55:59,3.0,145.92,Completed,8021.0,29.99,4.0,119.96
22,21.0,22.0,Elizabeth Moon,San Diego city,California,male,18 to 24 years,employed,bachelors_degree,production_transportation_material_moving,...,60797,161.0,2024-10-08 10:55:59,3.0,145.92,Completed,1273.0,11.99,2.0,23.98
23,21.0,22.0,Elizabeth Moon,San Diego city,California,male,18 to 24 years,employed,bachelors_degree,production_transportation_material_moving,...,60797,161.0,2024-10-08 10:55:59,3.0,145.92,Completed,3502.0,0.66,3.0,1.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,4984.0,54.0,Jessica Owens,San Jose city,California,male,25 to 34 years,employed,bachelors_degree,management_business_science_arts,...,76593,32.0,2024-08-09 21:25:03,2.0,172.73,Shipped,3788.0,9.59,3.0,28.77
5996,4984.0,54.0,Jessica Owens,San Jose city,California,male,25 to 34 years,employed,bachelors_degree,management_business_science_arts,...,76593,32.0,2024-08-09 21:25:03,2.0,172.73,Shipped,6524.0,35.99,4.0,143.96
6011,4999.0,6.0,Shane Wong,Tucson city,Arizona,male,18 to 24 years,employed,bachelors_degree,production_transportation_material_moving,...,35362,325.0,2024-05-12 12:41:55,3.0,40.69,Shipped,9496.0,7.79,1.0,7.79
6012,4999.0,6.0,Shane Wong,Tucson city,Arizona,male,18 to 24 years,employed,bachelors_degree,production_transportation_material_moving,...,35362,325.0,2024-05-12 12:41:55,3.0,40.69,Shipped,271.0,3.59,2.0,7.18


In [12]:
Products = Products.drop(columns = ['CreatedAt', 'SKU']).rename(columns = {'Name': 'ProductName', 'Price': 'ProductPrice'})
Products['ProductID'] = Products['ProductID'].astype(float)
Products['RetailerID'] = Products['RetailerID'].astype(float)

deneme = CustomerBuys.merge(Products,
                   how = 'left',
                   on = 'ProductID')

In [13]:
deneme

,CustomerID,RetailerID_x,CustomerName,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,...,Status,ProductID,Price,Quantity,TotalPrice,RetailerID_y,ProductName,MainCategory,SubCategory,ProductPrice
0,1.0,48.0,Courtney Bates,Los Angeles city,California,female,5 to 17 years,unemployed,less_than_high_school_diploma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,11.0,Alicia Silva,Nashville-Davidson metropolitan government (ba...,Tennessee,male,35 to 44 years,employed,some_college_or_associates_degree,management_business_science_arts,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,66.0,George Hernandez,Louisville/Jefferson County metro government (...,Kentucky,female,5 to 17 years,unemployed,less_than_high_school_diploma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,24.0,Nicholas Holloway,New York city,New York,female,75 years and over,employed,graduate_or_professional_degree,management_business_science_arts,...,Completed,5419.0,27.59,3.0,82.77,91.0,REMAXX Mini Portable Desktop Vacuum Cleaner Co...,appliances,All Appliances,27.59
4,5.0,38.0,Kelly Thomas,San Diego city,California,male,65 to 74 years,employed,less_than_high_school_diploma,sales_and_office_occupations,...,Completed,1440.0,23.99,4.0,95.96,24.0,Jewels Galaxy Jewellery For Women Stackable Ri...,stores,Amazon Fashion,23.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6010,4998.0,56.0,Pamela Hayes,Boston city,Massachusetts,female,18 to 24 years,employed,some_college_or_associates_degree,production_transportation_material_moving,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6011,4999.0,6.0,Shane Wong,Tucson city,Arizona,male,18 to 24 years,employed,bachelors_degree,production_transportation_material_moving,...,Shipped,9496.0,7.79,1.0,7.79,63.0,HomeStrap Innerwear/Undergarment Organizer 12+...,home & kitchen,Home Storage,7.79
6012,4999.0,6.0,Shane Wong,Tucson city,Arizona,male,18 to 24 years,employed,bachelors_degree,production_transportation_material_moving,...,Shipped,271.0,3.59,2.0,7.18,57.0,Vamsha Nature Care Air Purifying Money Plants ...,home & kitchen,Garden & Outdoors,3.59
6013,4999.0,6.0,Shane Wong,Tucson city,Arizona,male,18 to 24 years,employed,bachelors_degree,production_transportation_material_moving,...,Shipped,988.0,12.86,2.0,25.72,67.0,NEUTRON Royal Analog White and Orange Color Di...,accessories,Watches,12.86


In [14]:
Products[Products['ProductID'] == 5.0]

,ProductID,RetailerID,ProductName,MainCategory,SubCategory,ProductPrice
4,5.0,98.0,A2D? 11807. Dual Tone Shark Fin Shape Decorati...,car & motorbike,Car Electronics,7.19


In [15]:
CustomerBuys[CustomerBuys['ProductID'] == 5.0]

,CustomerID,RetailerID,CustomerName,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,...,Income,OrderID,OrderDate,Count,TotalAmount,Status,ProductID,Price,Quantity,TotalPrice
2657,2198.0,89.0,David Pollard,Detroit city,Michigan,female,55 to 64 years,employed,high_school_graduate,sales_and_office_occupations,...,25193,385.0,2024-06-26 21:18:15,3.0,112.74,Shipped,5.0,7.19,2.0,14.38


The above examples show that a product can be sold by different retailers. Thus, we don't need RetailID column in Products.

In [16]:
Products.drop(columns = ['RetailerID'], inplace = True)
CustomerBuys = CustomerBuys.merge(Products,
                                  how = 'left',
                                  on = 'ProductID')
CustomerBuys

,CustomerID,RetailerID,CustomerName,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,...,TotalAmount,Status,ProductID,Price,Quantity,TotalPrice,ProductName,MainCategory,SubCategory,ProductPrice
0,1.0,48.0,Courtney Bates,Los Angeles city,California,female,5 to 17 years,unemployed,less_than_high_school_diploma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,11.0,Alicia Silva,Nashville-Davidson metropolitan government (ba...,Tennessee,male,35 to 44 years,employed,some_college_or_associates_degree,management_business_science_arts,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,66.0,George Hernandez,Louisville/Jefferson County metro government (...,Kentucky,female,5 to 17 years,unemployed,less_than_high_school_diploma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,24.0,Nicholas Holloway,New York city,New York,female,75 years and over,employed,graduate_or_professional_degree,management_business_science_arts,...,82.77,Completed,5419.0,27.59,3.0,82.77,REMAXX Mini Portable Desktop Vacuum Cleaner Co...,appliances,All Appliances,27.59
4,5.0,38.0,Kelly Thomas,San Diego city,California,male,65 to 74 years,employed,less_than_high_school_diploma,sales_and_office_occupations,...,95.96,Completed,1440.0,23.99,4.0,95.96,Jewels Galaxy Jewellery For Women Stackable Ri...,stores,Amazon Fashion,23.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6010,4998.0,56.0,Pamela Hayes,Boston city,Massachusetts,female,18 to 24 years,employed,some_college_or_associates_degree,production_transportation_material_moving,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6011,4999.0,6.0,Shane Wong,Tucson city,Arizona,male,18 to 24 years,employed,bachelors_degree,production_transportation_material_moving,...,40.69,Shipped,9496.0,7.79,1.0,7.79,HomeStrap Innerwear/Undergarment Organizer 12+...,home & kitchen,Home Storage,7.79
6012,4999.0,6.0,Shane Wong,Tucson city,Arizona,male,18 to 24 years,employed,bachelors_degree,production_transportation_material_moving,...,40.69,Shipped,271.0,3.59,2.0,7.18,Vamsha Nature Care Air Purifying Money Plants ...,home & kitchen,Garden & Outdoors,3.59
6013,4999.0,6.0,Shane Wong,Tucson city,Arizona,male,18 to 24 years,employed,bachelors_degree,production_transportation_material_moving,...,40.69,Shipped,988.0,12.86,2.0,25.72,NEUTRON Royal Analog White and Orange Color Di...,accessories,Watches,12.86


In [17]:
FlagDf = CustomerBuys.groupby('CustomerID')['TotalAmount'].sum().reset_index()
FlagDf.rename(columns={'TotalAmount': 'PurchaseFlag'}, inplace=True)

# Create a flag: 1 if PurchaseFlag is greater than 0, else 0
FlagDf['PurchaseFlag'] = FlagDf['PurchaseFlag'].apply(lambda x: 1 if x > 0 else 0)

CustomerBuys = CustomerBuys.merge(FlagDf, on='CustomerID', how='left')
CustomerBuys.loc[CustomerBuys['PurchaseFlag'] == 0, ['Count', 'TotalAmount', 'Price', 'Quantity', 'TotalPrice']] = 0
CustomerBuys

,CustomerID,RetailerID,CustomerName,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,...,Status,ProductID,Price,Quantity,TotalPrice,ProductName,MainCategory,SubCategory,ProductPrice,PurchaseFlag
0,1.0,48.0,Courtney Bates,Los Angeles city,California,female,5 to 17 years,unemployed,less_than_high_school_diploma,NaN,...,NaN,NaN,0.00,0.0,0.00,NaN,NaN,NaN,NaN,0
1,2.0,11.0,Alicia Silva,Nashville-Davidson metropolitan government (ba...,Tennessee,male,35 to 44 years,employed,some_college_or_associates_degree,management_business_science_arts,...,NaN,NaN,0.00,0.0,0.00,NaN,NaN,NaN,NaN,0
2,3.0,66.0,George Hernandez,Louisville/Jefferson County metro government (...,Kentucky,female,5 to 17 years,unemployed,less_than_high_school_diploma,NaN,...,NaN,NaN,0.00,0.0,0.00,NaN,NaN,NaN,NaN,0
3,4.0,24.0,Nicholas Holloway,New York city,New York,female,75 years and over,employed,graduate_or_professional_degree,management_business_science_arts,...,Completed,5419.0,27.59,3.0,82.77,REMAXX Mini Portable Desktop Vacuum Cleaner Co...,appliances,All Appliances,27.59,1
4,5.0,38.0,Kelly Thomas,San Diego city,California,male,65 to 74 years,employed,less_than_high_school_diploma,sales_and_office_occupations,...,Completed,1440.0,23.99,4.0,95.96,Jewels Galaxy Jewellery For Women Stackable Ri...,stores,Amazon Fashion,23.99,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6010,4998.0,56.0,Pamela Hayes,Boston city,Massachusetts,female,18 to 24 years,employed,some_college_or_associates_degree,production_transportation_material_moving,...,NaN,NaN,0.00,0.0,0.00,NaN,NaN,NaN,NaN,0
6011,4999.0,6.0,Shane Wong,Tucson city,Arizona,male,18 to 24 years,employed,bachelors_degree,production_transportation_material_moving,...,Shipped,9496.0,7.79,1.0,7.79,HomeStrap Innerwear/Undergarment Organizer 12+...,home & kitchen,Home Storage,7.79,1
6012,4999.0,6.0,Shane Wong,Tucson city,Arizona,male,18 to 24 years,employed,bachelors_degree,production_transportation_material_moving,...,Shipped,271.0,3.59,2.0,7.18,Vamsha Nature Care Air Purifying Money Plants ...,home & kitchen,Garden & Outdoors,3.59,1
6013,4999.0,6.0,Shane Wong,Tucson city,Arizona,male,18 to 24 years,employed,bachelors_degree,production_transportation_material_moving,...,Shipped,988.0,12.86,2.0,25.72,NEUTRON Royal Analog White and Orange Color Di...,accessories,Watches,12.86,1


Transactions is not used because it shows only 100 unique OrderID where there are 1000 OrderID in the main dataset

BehavioralData is not used because of the below example. In the main dataset (CustomerBuys), we don't have any of the CustomerID&ProductID combination shown in BehavioralData dataset.

In [18]:
BehavioralData[BehavioralData['CustomerID'] == 4456.0]

,BehaviorID,CustomerID,ProductID,Timestamp,ClickSource,PageModule,ActionType,DwellTimeSeconds,DwellTimeCategory
456,457,4456,129,2024-08-07 01:06:35.378677,home,specification,view,211,t5
621,622,4456,9959,2024-08-04 12:02:35.380672,sale,bottom,purchase,219,t5
2095,2096,4456,8899,2024-08-06 16:50:35.399594,search,NaN,view,159,t5
3830,3831,4456,2010,2024-07-14 23:48:35.414554,cart,specification,view,70,t4
5460,5461,4456,4591,2024-07-23 06:13:35.429539,cart,comments,add_to_cart,54,t3
6303,6304,4456,272,2024-07-28 05:53:35.436521,home,bottom,add_to_cart,233,t5
6478,6479,4456,9045,2024-07-25 23:37:35.438515,category,NaN,purchase,197,t5
6675,6676,4456,4457,2024-08-05 16:15:35.439513,search,comments,purchase,284,t5
7271,7272,4456,9936,2024-07-25 16:29:35.444500,search,comments,purchase,256,t5


In [19]:
CustomerBuys[CustomerBuys['CustomerID'] == 4456.0]

,CustomerID,RetailerID,CustomerName,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,...,Status,ProductID,Price,Quantity,TotalPrice,ProductName,MainCategory,SubCategory,ProductPrice,PurchaseFlag
5338,4456.0,48.0,Courtney Gordon,Albuquerque city,New Mexico,male,25 to 34 years,employed,some_college_or_associates_degree,production_transportation_material_moving,...,Shipped,3139.0,59.99,1.0,59.99,T Teclusive Ldnio 2500W 10A Smart WiFi Extensi...,industrial supplies,Industrial & Scientific Supplies,59.99,1


Some dataframes are only keep key items and they can only be used for merging dataframes
- **ChannelsCampaigns:** StoreID, ChannelID
- **StoresCampaigns:** StoreID, CampaignID
- **AudiencesCampaigns:** AudienceID, CampaignID

# SELLERS

In [20]:
Stores.rename(columns = {'Name': 'StoreName', 'City': 'StoreCity', 'State': 'StoreState'}, inplace=True)

Stores.drop(columns = ['Country'], inplace=True) # only USA

SellerSells = Stores.merge(ChannelsCampaigns, 
                      how = 'left', 
                      on = 'StoreID').merge(StoresCampaigns, 
                                            how = 'left', 
                                            on = 'StoreID').merge(AudiencesCampaigns, 
                                                                  how = 'left', 
                                                                  on = 'CampaignID').drop(columns = ['CreatedAt'])
SellerSells

,StoreID,RetailerID,StoreName,StoreCity,StoreState,channelID,CampaignID,audienceID
0,1,57,Store 57,Los Angeles city,California,42.0,60.0,64.0
1,1,57,Store 57,Los Angeles city,California,42.0,60.0,114.0
2,1,57,Store 57,Los Angeles city,California,42.0,60.0,124.0
3,1,57,Store 57,Los Angeles city,California,42.0,60.0,132.0
4,1,57,Store 57,Los Angeles city,California,42.0,60.0,175.0
...,...,...,...,...,...,...,...,...
1252,100,81,Store 81,Los Angeles city,California,32.0,94.0,63.0
1253,100,81,Store 81,Los Angeles city,California,32.0,94.0,230.0
1254,100,81,Store 81,Los Angeles city,California,60.0,94.0,16.0
1255,100,81,Store 81,Los Angeles city,California,60.0,94.0,63.0


By adding the keys, we can now use the information in Retailers, Audiences, Channels.

However, Retailers dataframe doesn't give us information about customer segmentation. (RetailerID, Name, ContactInfo, CreatedAt, UserID). 

We can only use this dataframe to use UserID as a key to merge with Users dataframe but Users also doesn't have any necessary information (UserID, Username, Email, PasswordHash, CreatedAt).

Therefore, we will only merge Audiences and Channels.

In [21]:
Audiences['AudienceID'] = Audiences['AudienceID'].astype(float)
Audiences = Audiences.drop(columns = ['CreatedAt', 'CriteriaValue'])
Audiences.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   AudienceID    250 non-null    float64
 1   RetailerID    250 non-null    int64  
 2   Insight       250 non-null    object 
 3   CriteriaType  172 non-null    object 
dtypes: float64(1), int64(1), object(2)
memory usage: 7.9+ KB


**ASSUMPTION:** Audiences df has both RetailerID and AudienceID, but when merging this df with others, only AudienceID is used. The primary analysis is to be centered around how different audiences engage, meaning AudienceID is the core key.

In [22]:
SellerSells = SellerSells.merge(Audiences.drop(columns = ['RetailerID']),
                                how = 'left', 
                                left_on = 'audienceID', 
                                right_on = 'AudienceID').drop(columns = ['AudienceID'])
SellerSells.head()

,StoreID,RetailerID,StoreName,StoreCity,StoreState,channelID,CampaignID,audienceID,Insight,CriteriaType
0,1,57,Store 57,Los Angeles city,California,42.0,60.0,64.0,channel preference,video
1,1,57,Store 57,Los Angeles city,California,42.0,60.0,114.0,geographic,city
2,1,57,Store 57,Los Angeles city,California,42.0,60.0,124.0,promotion preference,NaN
3,1,57,Store 57,Los Angeles city,California,42.0,60.0,132.0,channel preference,website and blogs
4,1,57,Store 57,Los Angeles city,California,42.0,60.0,175.0,purchase propensity,completed


Add campaign information

In [23]:
MarketingCampaigns['StartDate'] = pd.to_datetime(MarketingCampaigns['StartDate'])
MarketingCampaigns['EndDate'] = pd.to_datetime(MarketingCampaigns['EndDate'])

MarketingCampaigns.head()

,CampaignID,Name,StartDate,EndDate,Budget,CreatedAt
0,1,Campaign A,2024-08-08 21:50:33.556668,2024-09-07 21:50:33.557010,7626.44,2024-08-08 21:50:33.558220
1,2,Campaign B,2024-08-07 21:50:33.556719,2024-09-06 21:50:33.557013,3690.77,2024-08-07 21:50:33.558227
2,3,Campaign C,2024-08-06 21:50:33.556762,2024-09-05 21:50:33.557017,5664.16,2024-08-06 21:50:33.558232
3,4,Campaign D,2024-08-05 21:50:33.556768,2024-09-04 21:50:33.557020,5727.09,2024-08-05 21:50:33.558234
4,5,Campaign E,2024-08-04 21:50:33.556771,2024-09-03 21:50:33.557022,6056.04,2024-08-04 21:50:33.558237


In [24]:
MarketingCampaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   CampaignID  100 non-null    int64         
 1   Name        100 non-null    object        
 2   StartDate   100 non-null    datetime64[ns]
 3   EndDate     100 non-null    datetime64[ns]
 4   Budget      100 non-null    float64       
 5   CreatedAt   100 non-null    object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(2)
memory usage: 4.8+ KB


In [25]:
# All campaigns start and end at 21:50, so the time is redundant
MarketingCampaigns['StartTime'] = MarketingCampaigns['StartDate'].dt.time

# Check
MarketingCampaigns['StartTime'].sort_values() # ascending

0     21:50:33.556668
1     21:50:33.556719
2     21:50:33.556762
3     21:50:33.556768
4     21:50:33.556771
           ...       
95    21:50:33.556995
96    21:50:33.556997
97    21:50:33.556999
98    21:50:33.557002
99    21:50:33.557004
Name: StartTime, Length: 100, dtype: object

In [26]:
# keep only dates and remove times from StartDate and EndDate
MarketingCampaigns['StartDate'] = MarketingCampaigns['StartDate'].dt.date
MarketingCampaigns['EndDate'] = MarketingCampaigns['EndDate'].dt.date

# Reconvert to datetime
MarketingCampaigns['StartDate'] = pd.to_datetime(MarketingCampaigns['StartDate'])
MarketingCampaigns['EndDate'] = pd.to_datetime(MarketingCampaigns['EndDate'])

# Estimate how many days a campaign takes
MarketingCampaigns['CampaignDuration'] = (MarketingCampaigns['EndDate'] - MarketingCampaigns['StartDate']).dt.days
MarketingCampaigns

,CampaignID,Name,StartDate,EndDate,Budget,CreatedAt,StartTime,CampaignDuration
0,1,Campaign A,2024-08-08,2024-09-07,7626.44,2024-08-08 21:50:33.558220,21:50:33.556668,30
1,2,Campaign B,2024-08-07,2024-09-06,3690.77,2024-08-07 21:50:33.558227,21:50:33.556719,30
2,3,Campaign C,2024-08-06,2024-09-05,5664.16,2024-08-06 21:50:33.558232,21:50:33.556762,30
3,4,Campaign D,2024-08-05,2024-09-04,5727.09,2024-08-05 21:50:33.558234,21:50:33.556768,30
4,5,Campaign E,2024-08-04,2024-09-03,6056.04,2024-08-04 21:50:33.558237,21:50:33.556771,30
...,...,...,...,...,...,...,...,...
95,96,Campaign R,2024-05-05,2024-06-04,9974.80,2024-05-05 21:50:33.558513,21:50:33.556995,30
96,97,Campaign S,2024-05-04,2024-06-03,5879.82,2024-05-04 21:50:33.558515,21:50:33.556997,30
97,98,Campaign T,2024-05-03,2024-06-02,8648.25,2024-05-03 21:50:33.558518,21:50:33.556999,30
98,99,Campaign U,2024-05-02,2024-06-01,1965.51,2024-05-02 21:50:33.558520,21:50:33.557002,30


Calculating 'WeekOfMonth'
- Subtracting 1 converts the day of the month into a zero-based index
- Dividing by 7 gives the zero-based week number
- Adding 1 converts the zero-based week index back to a one-based index

In [27]:
# there might be seasonal effects on campaigns
MarketingCampaigns['Month'] = MarketingCampaigns['StartDate'].dt.month
MarketingCampaigns['WeekOfMonth'] = (MarketingCampaigns['StartDate'].dt.day - 1) // 7 + 1

MarketingCampaigns

,CampaignID,Name,StartDate,EndDate,Budget,CreatedAt,StartTime,CampaignDuration,Month,WeekOfMonth
0,1,Campaign A,2024-08-08,2024-09-07,7626.44,2024-08-08 21:50:33.558220,21:50:33.556668,30,8,2
1,2,Campaign B,2024-08-07,2024-09-06,3690.77,2024-08-07 21:50:33.558227,21:50:33.556719,30,8,1
2,3,Campaign C,2024-08-06,2024-09-05,5664.16,2024-08-06 21:50:33.558232,21:50:33.556762,30,8,1
3,4,Campaign D,2024-08-05,2024-09-04,5727.09,2024-08-05 21:50:33.558234,21:50:33.556768,30,8,1
4,5,Campaign E,2024-08-04,2024-09-03,6056.04,2024-08-04 21:50:33.558237,21:50:33.556771,30,8,1
...,...,...,...,...,...,...,...,...,...,...
95,96,Campaign R,2024-05-05,2024-06-04,9974.80,2024-05-05 21:50:33.558513,21:50:33.556995,30,5,1
96,97,Campaign S,2024-05-04,2024-06-03,5879.82,2024-05-04 21:50:33.558515,21:50:33.556997,30,5,1
97,98,Campaign T,2024-05-03,2024-06-02,8648.25,2024-05-03 21:50:33.558518,21:50:33.556999,30,5,1
98,99,Campaign U,2024-05-02,2024-06-01,1965.51,2024-05-02 21:50:33.558520,21:50:33.557002,30,5,1


In [28]:
# All campaigns take 30 days so the duration is redundant
MarketingCampaigns[MarketingCampaigns['CampaignDuration'] != 30]

# remove unnecessary columns
MarketingCampaigns = MarketingCampaigns.drop(columns = ['CreatedAt', 'StartTime', 'CampaignDuration', 'StartDate', 'EndDate'])
MarketingCampaigns['CampaignID'] = MarketingCampaigns['CampaignID'].astype(float)
MarketingCampaigns.rename(columns={'Name': 'CampaignName'}, inplace=True)
MarketingCampaigns

,CampaignID,CampaignName,Budget,Month,WeekOfMonth
0,1.0,Campaign A,7626.44,8,2
1,2.0,Campaign B,3690.77,8,1
2,3.0,Campaign C,5664.16,8,1
3,4.0,Campaign D,5727.09,8,1
4,5.0,Campaign E,6056.04,8,1
...,...,...,...,...,...
95,96.0,Campaign R,9974.80,5,1
96,97.0,Campaign S,5879.82,5,1
97,98.0,Campaign T,8648.25,5,1
98,99.0,Campaign U,1965.51,5,1


ResultID in CampaignResults is not a key since it's not used in any other dataframe

In [29]:
CampaignResults = CampaignResults.drop(columns = ['Timestamp', 'ResultID'])
CampaignResults['CampaignID'] = CampaignResults['CampaignID'].astype(float)
CampaignResults

,CampaignID,Metric,Value
0,46.0,Clicks,2986
1,100.0,Impressions,709
2,12.0,Conversions,8833
3,32.0,Impressions,9195
4,39.0,Conversions,2928
...,...,...,...
95,97.0,Conversions,3143
96,100.0,Clicks,4512
97,43.0,Conversions,2300
98,8.0,Clicks,8355


In [30]:
SellerSells = SellerSells.merge(MarketingCampaigns, 
                                how = 'left', 
                                on = 'CampaignID').merge(CampaignResults, 
                                                         how = 'left', 
                                                         on = 'CampaignID')
SellerSells.head()

,StoreID,RetailerID,StoreName,StoreCity,StoreState,channelID,CampaignID,audienceID,Insight,CriteriaType,CampaignName,Budget,Month,WeekOfMonth,Metric,Value
0,1,57,Store 57,Los Angeles city,California,42.0,60.0,64.0,channel preference,video,Campaign H,7225.58,6.0,2.0,Conversions,6868.0
1,1,57,Store 57,Los Angeles city,California,42.0,60.0,114.0,geographic,city,Campaign H,7225.58,6.0,2.0,Conversions,6868.0
2,1,57,Store 57,Los Angeles city,California,42.0,60.0,124.0,promotion preference,NaN,Campaign H,7225.58,6.0,2.0,Conversions,6868.0
3,1,57,Store 57,Los Angeles city,California,42.0,60.0,132.0,channel preference,website and blogs,Campaign H,7225.58,6.0,2.0,Conversions,6868.0
4,1,57,Store 57,Los Angeles city,California,42.0,60.0,175.0,purchase propensity,completed,Campaign H,7225.58,6.0,2.0,Conversions,6868.0


In [31]:
Channels.rename(columns={'name': 'ChannelName'}, inplace=True)
Channels['channelID'] = Channels['channelID'].astype(float)
Channels.drop(columns = ['level', 'createdAt'], inplace = True)


SellerSells = SellerSells.merge(Channels, 
                                how = 'left', 
                                on = 'channelID')
SellerSells.head()

,StoreID,RetailerID,StoreName,StoreCity,StoreState,channelID,CampaignID,audienceID,Insight,CriteriaType,CampaignName,Budget,Month,WeekOfMonth,Metric,Value,ChannelName,type
0,1,57,Store 57,Los Angeles city,California,42.0,60.0,64.0,channel preference,video,Campaign H,7225.58,6.0,2.0,Conversions,6868.0,Channel P1,email
1,1,57,Store 57,Los Angeles city,California,42.0,60.0,114.0,geographic,city,Campaign H,7225.58,6.0,2.0,Conversions,6868.0,Channel P1,email
2,1,57,Store 57,Los Angeles city,California,42.0,60.0,124.0,promotion preference,NaN,Campaign H,7225.58,6.0,2.0,Conversions,6868.0,Channel P1,email
3,1,57,Store 57,Los Angeles city,California,42.0,60.0,132.0,channel preference,website and blogs,Campaign H,7225.58,6.0,2.0,Conversions,6868.0,Channel P1,email
4,1,57,Store 57,Los Angeles city,California,42.0,60.0,175.0,purchase propensity,completed,Campaign H,7225.58,6.0,2.0,Conversions,6868.0,Channel P1,email


In [32]:
CustomerBuys.head()

,CustomerID,RetailerID,CustomerName,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,...,Status,ProductID,Price,Quantity,TotalPrice,ProductName,MainCategory,SubCategory,ProductPrice,PurchaseFlag
0,1.0,48.0,Courtney Bates,Los Angeles city,California,female,5 to 17 years,unemployed,less_than_high_school_diploma,NaN,...,NaN,NaN,0.00,0.0,0.00,NaN,NaN,NaN,NaN,0
1,2.0,11.0,Alicia Silva,Nashville-Davidson metropolitan government (ba...,Tennessee,male,35 to 44 years,employed,some_college_or_associates_degree,management_business_science_arts,...,NaN,NaN,0.00,0.0,0.00,NaN,NaN,NaN,NaN,0
2,3.0,66.0,George Hernandez,Louisville/Jefferson County metro government (...,Kentucky,female,5 to 17 years,unemployed,less_than_high_school_diploma,NaN,...,NaN,NaN,0.00,0.0,0.00,NaN,NaN,NaN,NaN,0
3,4.0,24.0,Nicholas Holloway,New York city,New York,female,75 years and over,employed,graduate_or_professional_degree,management_business_science_arts,...,Completed,5419.0,27.59,3.0,82.77,REMAXX Mini Portable Desktop Vacuum Cleaner Co...,appliances,All Appliances,27.59,1
4,5.0,38.0,Kelly Thomas,San Diego city,California,male,65 to 74 years,employed,less_than_high_school_diploma,sales_and_office_occupations,...,Completed,1440.0,23.99,4.0,95.96,Jewels Galaxy Jewellery For Women Stackable Ri...,stores,Amazon Fashion,23.99,1


In [34]:
df = CustomerBuys.merge(SellerSells, 
                        how = 'left', 
                        on = 'RetailerID')
df

,CustomerID,RetailerID,CustomerName,CustomerCity,CustomerState,Gender,Age,EmploymentStatus,Education,Occupation,...,Insight,CriteriaType,CampaignName,Budget,Month,WeekOfMonth,Metric,Value,ChannelName,type
0,1.0,48.0,Courtney Bates,Los Angeles city,California,female,5 to 17 years,unemployed,less_than_high_school_diploma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,11.0,Alicia Silva,Nashville-Davidson metropolitan government (ba...,Tennessee,male,35 to 44 years,employed,some_college_or_associates_degree,management_business_science_arts,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,66.0,George Hernandez,Louisville/Jefferson County metro government (...,Kentucky,female,5 to 17 years,unemployed,less_than_high_school_diploma,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,24.0,Nicholas Holloway,New York city,New York,female,75 years and over,employed,graduate_or_professional_degree,management_business_science_arts,...,interest based,NaN,Campaign Q,6548.08,7.0,4.0,Conversions,8892.0,Channel O2,podcast
4,4.0,24.0,Nicholas Holloway,New York city,New York,female,75 years and over,employed,graduate_or_professional_degree,management_business_science_arts,...,interest based,NaN,Campaign Q,6548.08,7.0,4.0,Clicks,3438.0,Channel O2,podcast
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115151,5000.0,3.0,Melissa Martin,Houston city,Texas,female,45 to 54 years,employed,some_college_or_associates_degree,management_business_science_arts,...,engagement status,dwell_time_seconds,Campaign C,4520.77,7.0,2.0,Impressions,1472.0,Channel A1,social media
115152,5000.0,3.0,Melissa Martin,Houston city,Texas,female,45 to 54 years,employed,some_college_or_associates_degree,management_business_science_arts,...,purchase propensity,pending,Campaign C,4520.77,7.0,2.0,Clicks,2912.0,Channel A1,social media
115153,5000.0,3.0,Melissa Martin,Houston city,Texas,female,45 to 54 years,employed,some_college_or_associates_degree,management_business_science_arts,...,purchase propensity,pending,Campaign C,4520.77,7.0,2.0,Impressions,1472.0,Channel A1,social media
115154,5000.0,3.0,Melissa Martin,Houston city,Texas,female,45 to 54 years,employed,some_college_or_associates_degree,management_business_science_arts,...,geographic,city,Campaign C,4520.77,7.0,2.0,Clicks,2912.0,Channel A1,social media


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115156 entries, 0 to 115155
Data columns (total 43 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   CustomerID        115156 non-null  float64
 1   RetailerID        115156 non-null  float64
 2   CustomerName      115156 non-null  object 
 3   CustomerCity      115156 non-null  object 
 4   CustomerState     115156 non-null  object 
 5   Gender            115156 non-null  object 
 6   Age               115156 non-null  object 
 7   EmploymentStatus  115156 non-null  object 
 8   Education         115156 non-null  object 
 9   Occupation        82683 non-null   object 
 10  Industry          82683 non-null   object 
 11  Income            115156 non-null  int64  
 12  OrderID           35826 non-null   float64
 13  OrderDate         35826 non-null   object 
 14  Count             115156 non-null  float64
 15  TotalAmount       115156 non-null  float64
 16  Status            35